In [3]:
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint
import re
from sklearn.manifold import TSNE
import numpy as np
import streamlit as st
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
filepath = 'warpeace.txt'
# Load the text
with open(filepath, 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# Preprocess the text
text = text.lower()  # Convert to lowercase
text = re.sub(r'[^a-z\s\.]', '', text)  # Remove special characters except full stops
words = text.split()  # Split into words
words = [word for word in words if word]  # Remove empty strings

# Remove words with less than 2 characters (for this example)
words = [word for word in words if len(word) > 1]
print(len(words))
words = pd.Series(words)

546565


In [7]:
unique_words = sorted(set(words))
stoi = {s: i + 1 for i, s in enumerate(unique_words)}  # Map words to indices
stoi['.'] = 0  # End-of-sentence token
itos = {i: s for s, i in stoi.items()}

# Prepare input-output pairs
block_size = 5  # Number of words to use as context
X, Y = [], []

for i in range(len(words) - block_size):
    context = [0] * block_size  # Start with the end-of-sentence token
    for j in range(block_size):
        context[j] = stoi[words[i + j]]
    X.append(context)
    Y.append(stoi[words[i + block_size]])# The next word to predict

print(len(X))
print(len(Y))

546560
546560


In [8]:
X = torch.tensor(X, dtype=torch.long)  # Input tensor
Y = torch.tensor(Y, dtype=torch.long)  # Output tensor

In [9]:
# Define model parameters
emb_dim = 64  # Larger embedding size for words
hidden_size = 1024  # Larger hidden layer size

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)  # Flatten the embeddings
        x = F.tanh(self.lin1(x))  # Use tanh activation
        x = self.lin2(x)
        return x

# Initialize the model
model = NextWord(block_size, len(stoi), emb_dim, hidden_size).to(device)
model = model.to(device)

In [10]:
def generate_word(model, itos, stoi, block_size, max_len=10):
    context = [0] * block_size  # Start with end-of-sentence token
    words_generated = []
    
    for _ in range(max_len):
        x = torch.tensor(context).view(1, -1).to(device)
        y_pred = model(x)
        ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
        word = itos[ix]
        if word == '.':
            break
        words_generated.append(word)
        context = context[1:] + [ix]  # Update context

    return ' '.join(words_generated)

In [15]:
block_size = 5
try:
    mcopy = torch.load('model-Emb64-Con5-Tanh.pth')
except:
    mcopy = torch.load('model-Emb64-Con5-Tanh.pth',map_location=device)
mcopy.eval()
# Function to preprocess the input sentence
def preprocess_input(sentence, stoi, block_size):
    sentence = sentence.lower()  # Convert to lowercase
    sentence = re.sub(r'[^a-z\s\.]', '', sentence)  # Remove special characters except full stops
    words = sentence.split()  # Split into words
    words = [word for word in words if word]  # Remove empty strings

    # Create context from the last block_size words
    context = [0] * block_size  # Start with the end-of-sentence token
    for i in range(block_size):
        if i < len(words):
            word = words[i]
            context[i] = stoi.get(word, 0)  # Get the index, use 0 if word not in vocab
        else:
            context[i] = 0  # Fill with end-of-sentence token if fewer words

    return context

# Input sentence
input_sentence = "The men fought very bravely"

# Preprocess the input sentence
context = preprocess_input(input_sentence, stoi, block_size)

# Convert context to tensor and send to device
context_tensor = torch.tensor(context, dtype=torch.long).view(1, -1).to(device)

# Generate the next word
with torch.no_grad():  # Disable gradient calculation for inference
    y_pred = mcopy(context_tensor)  # Forward pass
    ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()  # Sample from the distribution
    next_word = itos[ix]  # Convert index to word

print(f"The predicted next word for the input '{input_sentence}' is: '{next_word}'")

The predicted next word for the input 'The men fought very bravely' is: 'from'


In [ ]:
def generate_sentence(model, context, itos, stoi, block_size, max_len=10):
    context = preprocess_input(context, stoi, block_size)
    words_generated = []
    
    for _ in range(max_len):
        x = torch.tensor(context).view(1, -1).to(device)
        y_pred = model(x)
        ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
        word = itos[ix]
        if word == '.':
            break
        words_generated.append(word)
        context = context[1:] + [ix]  # Update context

    return ' '.join(words_generated)

In [19]:
%%writefile streamlit_app.py

import streamlit as st

# Streamlit App
st.title('Next Word Prediction')
st.write('This app predicts the next word in a sentence using a simple neural network model trained on the text of War and Peace.')

# Input fields
input_sentence = st.text_input('Enter initial text for prediction:', placeholder="e.g., 'It was a bright cold day in April'")
contextLength = st.selectbox("Context Length", [5, 10])
embeddingDim = st.selectbox("Embedding Dimension", [64, 128])
activation = st.selectbox("Activation Function", ['tanh', 'ReLU'])
maxLen = st.number_input('Max Length of Prediction', 1, 20, 1)
randomSeed = st.number_input('Random Seed', value=42)

# Prediction Button
if st.button('Predict Next Word/Letter'):
    try:
        # Placeholder for demonstration; replace with actual model prediction
        ans = "Example prediction text"  
        st.write(f"Generated Text: {ans}")
    except Exception as e:
        st.error(f"An error occurred: {e}")


Writing streamlit_app.py


In [20]:
!streamlit run streamlit_app.py

^C
